Read data

In [ ]:
#import nvidia.cudnn
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print("Shape x_train:")
print(x_train.shape)
print()
print("Number of dimensions:")
print(x_train.ndim)
print()
print("Datatypes:")
print(x_train.dtype)
print()
print("Shape x_test:")
print(x_test.shape)


## Visualisize images
# read call lables
class_names = np.unique(y_train)

# Set up the figure for a 10x10 plot
fig, axes = plt.subplots(10, 10, figsize=(15, 15))

for i in range(10):
    # Get indices for images of class i
    indices = np.where(y_train == i)[0]
    # Randomly select 10 indices from the class
    random_indices = np.random.choice(indices, size=10, replace=False)
    
    for j, idx in enumerate(random_indices):
        # Plot the image in the appropriate subplot
        axes[i, j].imshow(x_train[idx])
        axes[i, j].axis('off')  # Hide axis ticks
        if j == 0:
            axes[i, j].set_ylabel(class_names[i], fontsize=12)  # Label the class

plt.subplots_adjust(wspace=0.1, hspace=0.3)
plt.show()

# converts the data type of the x_train array to float32
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print('After reshaping:')

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print(x_train.shape)
print(x_train.ndim)
print(x_train.dtype)
print(x_test.shape)

# converts target to categorical data
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print(y_train.shape)
print(y_test.shape)